# 📌 Uygulama – 4  
# 🧠 “TorchScript CNN + Tek Görüntüde C++/LibTorch Inference”  

Bu uygulamada artık **basit preprocess** değil, tamamen **bir CNN modelini C++ tarafında çalıştırma** aşamasına geçtim.  
Python’da eğittiğim CNN modelini TorchScript’e çevirdim ve C++’ta tek bir resim üzerinde inference yaptım.

---

## 🎯 1) Amaç — *Bu projede ne yaptım?*

Bu uygulamanın amacı:

> **Python’da eğittiğim CNN modelini (CIFAR tarzı), TorchScript olarak C++’ta yüklemek,  
> OpenCV ile resim okuyup preprocess etmek ve softmax + argmax ile sınıf tahmini yapmaktı.**

Burada ilk kez:

- Gerçek eğitimli bir modeli yükledim  
- Preprocess pipeline’ı tam uyguladım  
- Modelden gerçek sınıf tahmini aldım  
- Forward → logits → softmax → argmax zincirini kurdum

---

## 🧩 2) Ana Adımlar — *Kodda yaptığım işlemler*

### 🟦 0) Model dosyasını kontrol ettim

```cpp
std::ifstream f("model_ts.pt");
```
* Exe ile aynı klasörde değilse → hata

* Bu C++ tarafında model yükleme için kritik bir kontrol

### 🟥 1) TorchScript modeli yükledim
```cpp
torch::jit::script::Module module = torch::jit::load(model_path);
module.to(torch::kCPU);
module.eval();
```


* TorchScript formatındaki CNN modelini başarıyla aldım

* CPU’ya taşıdım

* eval() moduna aldım → dropout ve batchnorm sabitleniyor

### 🟩 2) test.jpg görselini yükledim
```cpp
cv::Mat img = cv::imread("test.jpg");
```


* Dosya yoksa yine hata

* Resmin orijinal boyutunu ekrana bastım

### 🟦 3) Görseli 64×64 boyutuna resize ettim
```cpp
cv::resize(img, img, cv::Size(64, 64));
```


* Modelin eğitim sırasında kullandığım input boyutu → 64×64

* Resmi modele uygun hale getirdim

### 🟧 4) BGR → RGB dönüşümü
```cpp
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
```


* OpenCV = BGR

* PyTorch = RGB

Kanal sırasını düzelttim

### 🟨 5) uint8 → float32 + normalize
```cpp
img.convertTo(img_float, CV_32F, 1.0 / 255.0);
```


* 0–255 → 0–1

* Veri tipi: uint8 → float32

* CNN için standart preprocess

### 🟫 6) HWC → Tensor (NHWC)
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,
    {1, img_float.rows, img_float.cols, 3},
    torch::TensorOptions().dtype(torch::kFloat32)
);
```


Tensörün ilk hali:
* [1, 64, 64, 3] → NHWC formatı

### 🟪 7) NHWC → NCHW + clone()
```cpp
tensor_image = tensor_image.permute({0,3,1,2}).contiguous();
tensor_image = tensor_image.clone();
```

* PyTorch modelleri NCHW beklediği için permute yaptım

* contiguous() → bellek hizalaması

* clone() → tensör artık OpenCV’den bağımsız

Final input shape:
* [1, 3, 64, 64]

### 🟩 8) Forward() çalıştırdım
```cpp
std::vector<torch::jit::IValue> inputs;
inputs.push_back(tensor_image);
torch::Tensor output = module.forward(inputs).toTensor();
```


* IValue listesine tensörü ekledim

* Modeli çalıştırdım

* Output shape: [1, 10]

### 🟧 9) Softmax + Argmax ile sınıf tahmini yaptım
```cpp
torch::Tensor probs = torch::softmax(output, 1);
torch::Tensor pred_class = probs.argmax(1);
```


* softmax → sınıf olasılıkları

* argmax → en yüksek olasılıklı sınıf index’i

Final çıktı:

✔ logits

✔ probs

✔ predicted class

---
## 🧠 3) Bu projede öğrendiğim temel kavramlar

📌 TorchScript modelini C++’ta yükleme

📌 Eval modunda inference çalıştırma

📌 cv::Mat → torch::Tensor (full preprocess pipeline)

📌 Resize, RGB dönüşümü, normalize

📌 NHWC → NCHW reshape mantığı

📌 IValue ile forward input hazırlama

📌 Softmax + Argmax ile sınıf tahmini alma

📌 clone() ile bellek bağımsızlığı sağlama

📌 C++/LibTorch ile gerçek CNN modeli çalıştırma akışı

Bu uygulama, LibTorch tarafında “gerçek model inference” seviyesine yükseldiğim aşama oldu.

---
## ⚠️ 4) Dikkat edilmesi gereken noktalar

model_ts.pt EXE ile aynı klasörde olmalı

Input tensörü NCHW olmalı (aksi halde şekil hatası)

Görsel mutlaka 64×64’e resize edilmeli

Normalize edilmezse model saçma sonuç verir

from_blob tensörü OpenCV belleğine bağlar
→ o yüzden preprocess sonunda clone() zorunlu

Forward çağrısından önce model eval() modunda olmalı

---
## 🔖 5) Etiketler / Kategori

TorchScript Load

CNN Inference

Preprocess Pipeline

Single Image Prediction

LibTorch + OpenCV

Softmax & Argmax

---

## 📝 6) Tek Cümlelik Özet

Eğittiğim CNN modelini TorchScript olarak C++’ta yükledim, OpenCV ile preprocess ettim ve softmax + argmax kullanarak tek bir görüntü üzerinde sınıf tahmini gerçekleştirdi